In [1]:
Datenbank vorbereiten
 - Indexierung erstellen

SyntaxError: invalid syntax (3725173021.py, line 1)

In [ ]:
import sqlite3

conn = sqlite3.connect('data/SOLKAT_DACH.gpkg')
cursor = conn.cursor()

index_name = "GWR_EGID_INDEX"

create_index_sql = "CREATE INDEX {0} ON SOLKAT_CH_DACH(GWR_EGID)".format(index_name)

cursor.execute(create_index_sql)

conn.commit()
conn.close()

Daten einlesen aus Building Datenbank

In [32]:
import sqlite3
import numpy as np
import pandas as pd

conn = sqlite3.connect('data/data.sqlite')                                
cursor = conn.cursor()

# query building DB
sql = "SELECT EGID, GGDENAME, GDEKT from building where GDEKT = 'AI';"

df_building = pd.read_sql_query(sql, conn)

print(df_building)

print(df_building.head())

# Verbindung schließen
conn.close()

#df_building.to_csv("data/EGID.csv", index=False)

          EGID       GGDENAME GDEKT
0      1712079      Appenzell    AI
1      1712080      Appenzell    AI
2      1712081      Appenzell    AI
3      1712082      Appenzell    AI
4      1712083      Appenzell    AI
...        ...            ...   ...
10233  9018426  Schwende-Rüte    AI
10234  9018427  Schwende-Rüte    AI
10235  9041168  Schwende-Rüte    AI
10236  9080869  Schwende-Rüte    AI
10237  9080934  Schwende-Rüte    AI

[10238 rows x 3 columns]
      EGID   GGDENAME GDEKT
0  1712079  Appenzell    AI
1  1712080  Appenzell    AI
2  1712081  Appenzell    AI
3  1712082  Appenzell    AI
4  1712083  Appenzell    AI


In [48]:
Daten einlesen aus Dach Datenbank

SyntaxError: invalid syntax (1531320668.py, line 1)

In [1]:
import sqlite3
import numpy as np
import pandas as pd

conn = sqlite3.connect('data/SOLKAT_DACH.gpkg')                                
cursor = conn.cursor()

sql =  """
    SELECT 
        SUM(FLAECHE) AS Summe_Flaeche, 
        SUM(STROMERTRAG) AS Summe_Stromertrag, 
        SUM(FLAECHE_KOLLEKTOREN) AS Summe_Flaeche_Kollektoren, 
        GWR_EGID AS EGID
    FROM 
        SOLKAT_CH_DACH
    WHERE
        GWR_EGID IS NOT NULL
    GROUP BY
        GWR_EGID
    LIMIT
        1000000;
    """

print(sql)

df_dach = pd.read_sql_query(sql, conn)

# Verbindung schließen
conn.close()

#Spalte EGID muss für merge gleicher Datentyp wie df_building/EGID haben
#df_dach["EGID"] = df_dach["EGID"].astype(str)
#
##Mergen und Zeilen behalten, die in beiden Dataframes gefunden wurden
#merged_df = pd.merge(df_building, df_dach, how='inner', on='EGID')
#
#print(merged_df.head())
#
##df_dach.to_csv("data/Dach.csv", index=False)
#merged_df.to_csv("data/DachInfo.csv", index=False)


    SELECT 
        SUM(FLAECHE) AS Summe_Flaeche, 
        SUM(STROMERTRAG) AS Summe_Stromertrag, 
        SUM(FLAECHE_KOLLEKTOREN) AS Summe_Flaeche_Kollektoren, 
        GWR_EGID AS EGID
    FROM 
        SOLKAT_CH_DACH
    WHERE
        GWR_EGID IS NOT NULL
    GROUP BY
        GWR_EGID
    LIMIT
        1000000;
    


In [34]:
import sqlite3
import numpy as np
import pandas as pd

conn = sqlite3.connect('data/SOLKAT_DACH.gpkg')
cursor = conn.cursor()

# Inhalt von Dataframe löschen
df_dach.drop(index=df_dach.index, inplace=True)

# Schleife über jede Gruppe von 500 EGIDs
for i in range(0, len(df_building), 500):
    # Aktuelle Gruppe von EGIDs auswählen
    egid_group = df_building["EGID"].iloc[i:i+500]
    
    # WHERE-Klausel für die Abfrage erstellen
    where_clause = "WHERE " + " OR ".join(["GWR_EGID='{}'".format(egid) for egid in egid_group])
    
    # SQL-Abfrage ausführen
    sql =  """
        SELECT 
            SUM(FLAECHE) AS Summe_Flaeche, 
            SUM(STROMERTRAG) AS Summe_Stromertrag, 
            SUM(FLAECHE_KOLLEKTOREN) AS Summe_Flaeche_Kollektoren, 
            GWR_EGID AS EGID
        FROM 
            SOLKAT_CH_DACH
        {}
        GROUP BY
            GWR_EGID;
        """.format(where_clause)
    
    result = pd.read_sql_query(sql, conn)

    # Ergebnis dem bestehenden DataFrame "df_dach" hinzufügen
    df_dach = df_dach.append(result, ignore_index=True)


# Verbindung schließen
conn.close()


# Spalte EGID muss für merge gleicher Datentyp wie df_building/EGID haben
df_dach["EGID"] = df_dach["EGID"].astype(str)

# Mergen und Zeilen behalten, die in beiden Dataframes gefunden wurden
df_merge = pd.merge(df_building, df_dach, how='inner', on='EGID')

df_merge.to_csv("data/DachInfo.csv", index=False)


C:\Users\flavi\AppData\Local\Temp\ipykernel_27804\2463110837.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result, ignore_index=True)
C:\Users\flavi\AppData\Local\Temp\ipykernel_27804\2463110837.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result, ignore_index=True)
C:\Users\flavi\AppData\Local\Temp\ipykernel_27804\2463110837.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result, ignore_index=True)
C:\Users\flavi\AppData\Local\Temp\ipykernel_27804\2463110837.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dach = df_dach.append(result,